# Notebook de la dimensión Mensajero - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [1]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2 y 3. Extracción y transgormación de datos desde la base de datos - Extract and Transform

In [2]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [3]:
# Consulta SQL a la base de datos OLTP - se hace proceso de transform en algunas columnas
query = """
SELECT 
    m.id AS id_mensajero,
    u.first_name || ' ' || u.last_name AS nombre_mensajero,
    m.activo,
    m.salario,
    m.ciudad_operacion_id AS id_ciudad,
    c.nombre AS nombre_ciudad,
    d.departamento_id AS id_departamento,
    d.nombre AS nombre_departamento
FROM clientes_mensajeroaquitoy m
JOIN auth_user u ON m.user_id = u.id
LEFT JOIN ciudad c ON m.ciudad_operacion_id = c.ciudad_id
LEFT JOIN departamento d ON c.departamento_id = d.departamento_id;
"""

# Ejecutar consulta
df_mensajero = pd.read_sql(query, engine_fuente)

In [6]:
from datetime import datetime

# Agregar columna de fecha de carga
df_mensajero["fecha_carga"] = datetime.now().date()

In [7]:
# Eliminar duplicados por seguridad
df_mensajero = df_mensajero.drop_duplicates(subset=['id_mensajero'])

In [8]:
# Verificar
print(df_mensajero.head())

   id_mensajero                    nombre_mensajero  activo    salario  \
0             8  pepito_el_rapido pepito_el_furioso    True  1160000.0   
1            10  pepito_el_rapido pepito_el_furioso    True  1160000.0   
2            11  pepito_el_rapido pepito_el_furioso    True  1160000.0   
3            12  pepito_el_rapido pepito_el_furioso    True  1160000.0   
4            13  pepito_el_rapido pepito_el_furioso    True  1160000.0   

   id_ciudad nombre_ciudad  id_departamento nombre_departamento fecha_carga  
0        2.0       PALMIRA              1.0     VALLE DEL CAUCA  2025-06-05  
1        1.0          CALI              1.0     VALLE DEL CAUCA  2025-06-05  
2        1.0          CALI              1.0     VALLE DEL CAUCA  2025-06-05  
3        1.0          CALI              1.0     VALLE DEL CAUCA  2025-06-05  
4        4.0         PASTO              4.0              NARIÑO  2025-06-05  


## 4. Guardado de la dimensión - Load

In [9]:
# Guardar como archivo CSV
df_mensajero.to_csv("dim_mensajero.csv", index=False)

In [10]:
# Cargar en la bodega de datos OLAP
df_mensajero.to_sql("dim_mensajero", engine_bodega, if_exists="replace", index=False)

50

In [11]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_mensajero.csv")
print(df_verificacion.head())

   id_mensajero                    nombre_mensajero  activo    salario  \
0             8  pepito_el_rapido pepito_el_furioso    True  1160000.0   
1            10  pepito_el_rapido pepito_el_furioso    True  1160000.0   
2            11  pepito_el_rapido pepito_el_furioso    True  1160000.0   
3            12  pepito_el_rapido pepito_el_furioso    True  1160000.0   
4            13  pepito_el_rapido pepito_el_furioso    True  1160000.0   

   id_ciudad nombre_ciudad  id_departamento nombre_departamento fecha_carga  
0        2.0       PALMIRA              1.0     VALLE DEL CAUCA  2025-06-05  
1        1.0          CALI              1.0     VALLE DEL CAUCA  2025-06-05  
2        1.0          CALI              1.0     VALLE DEL CAUCA  2025-06-05  
3        1.0          CALI              1.0     VALLE DEL CAUCA  2025-06-05  
4        4.0         PASTO              4.0              NARIÑO  2025-06-05  
